In [1]:
import time
import uuid
from typing import Optional, Dict, List

In [2]:
class ContextObject:
    def __init__(
        self, 
        role: Optional[str] = None, 
        content: Optional[str] = None, 
        name: Optional[str] = None,
        status: Optional[str] = None,      # 仅用于控制可见性/可编辑性
        model_tags: Optional[List[str]] = None,  # 新增：模型能力标签
        metadata: Optional[Dict] = None,
        id: Optional[str] = None,
        parent_id: Optional[str] = None
    ):
        self.role = role
        self.content = content
        self.name = name
        self.status = status
        self.model_tags = model_tags or []  # 默认空列表
        self.metadata = metadata or {}
        self.id = id
        self.parent_id = parent_id

    def to_dict(self) -> Dict:
        return {
            "role": self.role,
            "content": self.content,
            "name": self.name,
            "status": self.status,
            "model_tags": self.model_tags,  # 序列化新增字段
            "metadata": self.metadata,
            "id": self.id,
            "parent_id": self.parent_id
        }

    @classmethod
    def from_dict(cls, data: Dict) -> 'ContextObject':
        return cls(
            role=data.get("role"),
            content=data.get("content"),
            name=data.get("name"),
            status=data.get("status"),
            model_tags=data.get("model_tags", []),  # 反序列化新增字段
            metadata=data.get("metadata", {}),
            id=data.get("id"),
            parent_id=data.get("parent_id")
        )

In [3]:
class ExecutionEngine:

    def __init__(self):
        self.contexts: Dict[str, ContextObject] = {}  # 存储上下文对象，键为对象ID
    
    def add_context(self, context: ContextObject) -> str:
        if not context.id:
            context.id = str(uuid.uuid4())  # 生成唯一ID
        self.contexts[context.id] = context
        return context.id
    
    def execute(self, context_id: str) -> Optional[str]:
        context = self.contexts.get(context_id)
        if not context:
            print(f"Context with ID {context_id} not found.")
            return None
        
        if context.status == "interrupted":
            print(f"Waiting for context {context_id} to be ready...")
        elif context.status == "deferred":
            delay = context.metadata.get("defer_seconds", 5)  # 获取延迟时间，默认5秒
            print(f"Context {context_id} is deferred. Will execute after {delay} seconds.")
            time.sleep(delay)  # 模拟等待
            context.status = "active"
            print(f"Context {context_id} is now active.")
        elif context.status == "active":
            print(f"Context {context_id} is active. Executing immediately.")
        else:
            print(f"Context {context_id} has unknown status '{context.status}'. Executing with caution.")
    
    def resume(self, context_id: str):
        context = self.contexts.get(context_id)
        if context and context.status == "interrupted":
            context.status = "active"
            print(f"Context {context_id} has been resumed and is now active.")
        else:
            print(f"Context with ID {context_id} not found or not in interrupted state.")

In [4]:
# 示例流程
engine = ExecutionEngine()
context1 = ContextObject(role="user", content="Hello, how are you?", status="interrupted")
context_id1 = engine.add_context(context1)
engine.execute(context_id1)  # 输出等待信息
engine.resume(context_id1)   # 恢复上下文
engine.execute(context_id1)  # 输出执行信息

Waiting for context 2bd6c11d-8faa-4ee5-b332-279fdfe175cd to be ready...
Context 2bd6c11d-8faa-4ee5-b332-279fdfe175cd has been resumed and is now active.
Context 2bd6c11d-8faa-4ee5-b332-279fdfe175cd is active. Executing immediately.
